# DATA AUGMENTATION USING KERAS LAYERS

## Advantage -> Can be embedded into the model itself (Useful for Resizing and Rescaling)

In [17]:
# Import Dependencies
import import_ipynb
import Utils
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds
from tensorflow.keras.layers import Layer, Input, Normalization, Conv2D, MaxPooling2D, Dense, Flatten, BatchNormalization
from tensorflow.keras.metrics import BinaryAccuracy, FalsePositives, FalseNegatives, TrueNegatives, TruePositives, Precision, Recall, F1Score, AUC

In [10]:
# Augmentation Method Using Keras Sequential Layer
augmentLayers = tf.keras.Sequential([
    tf.keras.layers.RandomRotation(
        factor=(0.25, 0.2501),
    ),
    tf.keras.layers.RandomFlip(
        mode = 'HORIZONTAL',
    ),
    tf.keras.layers.Resizing(Utils.IMAGE_SIZE, Utils.IMAGE_SIZE),
    tf.keras.layers.Rescaling(1.0/255)
])
def augmentLayer(image, labels):
    return augmentLayers(image, training=True), labels

In [11]:
# Dataset Batching
BATCH_SIZE=32
BUFFER_SIZE = 8
trainDataset = Utils.trainDataset.map(augmentLayer).shuffle(buffer_size=BUFFER_SIZE, reshuffle_each_iteration=True).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
valDataset = Utils.valDataset.shuffle(buffer_size=BUFFER_SIZE, reshuffle_each_iteration=True).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
testDataset = Utils.testDataset.shuffle(buffer_size=BUFFER_SIZE, reshuffle_each_iteration=True).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

In [12]:
# Training
history = Utils.model.fit(trainDataset, validation_data=valDataset, epochs=5, verbose=1)

Epoch 1/5
689/689 ━━━━━━━━━━━━━━━━━━━━ 63s 91ms/step - auc: 0.8784 - binary_accuracy: 0.8284 - false_negatives: 515.0870 - false_positives: 1188.0044 - loss: 0.5217 - precision: 0.7940 - recall: 0.8906 - true_negatives: 4330.6567 - true_positives: 5022.2002 - val_auc: 0.5000 - val_binary_accuracy: 0.4962 - val_false_negatives: 1388.0000 - val_false_positives: 0.0000e+00 - val_loss: 3512.7798 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_true_negatives: 1367.0000 - val_true_positives: 0.0000e+00
Epoch 2/5
689/689 ━━━━━━━━━━━━━━━━━━━━ 62s 90ms/step - auc: 0.9435 - binary_accuracy: 0.8938 - false_negatives: 338.3087 - false_positives: 860.3478 - loss: 0.3498 - precision: 0.8605 - recall: 0.9416 - true_negatives: 4658.3506 - true_positives: 5198.9404 - val_auc: 0.5000 - val_binary_accuracy: 0.4929 - val_false_negatives: 1397.0000 - val_false_positives: 0.0000e+00 - val_loss: 7459.2158 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_true_negatives: 1358.0000 - val_tr

## Adding These Layers To The Model

In [15]:
# Using Sequential API
IMAGE_SIZE = 224
model = tf.keras.Sequential([
    Input(shape=(None, None, 3)),

    # Adding Resize/Rescale or Augment Layers Directly To The Model
    augmentLayers,

    Conv2D(filters=6, kernel_size=3, strides=1, padding='valid', activation='relu'),
    BatchNormalization(),
    MaxPooling2D(pool_size=2, strides=2),

    Conv2D(filters=16, kernel_size=3, strides=1, padding='valid', activation='relu'),
    BatchNormalization(),
    MaxPooling2D(pool_size=2, strides=2),

    Flatten(),

    Dense(100, activation='relu'),
    BatchNormalization(),
    Dense(10, activation='relu'),
    BatchNormalization(),
    Dense(1, activation='sigmoid'),
])
model.summary()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ sequential_3 (Sequential)       │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 222, 222, 6)    │           168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 222, 222, 6)    │            24 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 111, 111, 6)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 109, 109, 16)   │           880 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ (None, 109, 109, 16)   │            64 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 54, 54, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 46656)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 100)            │     4,665,700 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_6           │ (None, 100)            │           400 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 10)             │         1,010 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_7           │ (None, 10)             │            40 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │            11 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,668,297 (17.81 MB)

 Trainable params: 4,668,033 (17.81 MB)

 Non-trainable params: 264 (1.03 KB)

## Custom Keras Layer For Augmentation Using tf Methods

In [20]:
# Layer Class
class RotNinety(Layer):
    def __init__(self):
        super().__init__()
    def call(self, image):
        return tf.image.rot90(image)

In [21]:
# Augmentation Method Using Custom Layer
augmentLayers = tf.keras.Sequential([
    RotNinety(),
    tf.keras.layers.RandomFlip(
        mode = 'HORIZONTAL',
    ),
    tf.keras.layers.Resizing(Utils.IMAGE_SIZE, Utils.IMAGE_SIZE),
    tf.keras.layers.Rescaling(1.0/255)
])

In [22]:
# Using Sequential API
IMAGE_SIZE = 224
model = tf.keras.Sequential([
    Input(shape=(None, None, 3)),

    # Adding Resize/Rescale or Augment Layers Directly To The Model
    augmentLayers,

    Conv2D(filters=6, kernel_size=3, strides=1, padding='valid', activation='relu'),
    BatchNormalization(),
    MaxPooling2D(pool_size=2, strides=2),

    Conv2D(filters=16, kernel_size=3, strides=1, padding='valid', activation='relu'),
    BatchNormalization(),
    MaxPooling2D(pool_size=2, strides=2),

    Flatten(),

    Dense(100, activation='relu'),
    BatchNormalization(),
    Dense(10, activation='relu'),
    BatchNormalization(),
    Dense(1, activation='sigmoid'),
])
model.summary()

Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ sequential_6 (Sequential)       │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 222, 222, 6)    │           168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_8           │ (None, 222, 222, 6)    │            24 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 111, 111, 6)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 109, 109, 16)   │           880 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_9           │ (None, 109, 109, 16)   │            64 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 54, 54, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 46656)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 100)            │     4,665,700 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_10          │ (None, 100)            │           400 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 10)             │         1,010 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_11          │ (None, 10)             │            40 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 1)              │            11 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,668,297 (17.81 MB)

 Trainable params: 4,668,033 (17.81 MB)

 Non-trainable params: 264 (1.03 KB)